Implementating fem for truss using class

In [1]:
import math
import numpy as np
import copy

In [2]:
#import classes for nodes and elements
from node import node
from ele import ele

In [3]:
#define nodes along with boundary conditions
node1 = node(1,0,0,0,0,np.nan,np.nan)
node2 = node(2,2,0,0,0,np.nan,np.nan)
node3 = node(3,2,2,np.nan,np.nan,100000,-200000)

In [4]:
node1.print()
node2.print()
node3.print()

num =  1 
 pos_x = 0 
 pos_y =  0 
 dis_x =  0 
 dis_y =  0 
 f_x =  nan 
 f_y =  nan
num =  2 
 pos_x = 2 
 pos_y =  0 
 dis_x =  0 
 dis_y =  0 
 f_x =  nan 
 f_y =  nan
num =  3 
 pos_x = 2 
 pos_y =  2 
 dis_x =  nan 
 dis_y =  nan 
 f_x =  100000 
 f_y =  -200000


0

In [5]:
#define elements

ele1 = ele(1,1,1,1, 0 , 1 , 2)
ele2 = ele(2,1,1,1, math.pi / 2 , 2 , 3)
ele3 = ele(3,1,1,1, math.pi / 4 , 1 , 3)


In [6]:
ele1.print()
ele2.print()
ele3.print()

num =  1 
 length= 1 
 area =  1 
 youngs modulus =  1 
 theta =  0 
 node_a =  <class 'node.node'> 
 node_b =  <class 'node.node'>
num =  2 
 length= 1 
 area =  1 
 youngs modulus =  1 
 theta =  1.5707963267948966 
 node_a =  <class 'node.node'> 
 node_b =  <class 'node.node'>
num =  3 
 length= 1 
 area =  1 
 youngs modulus =  1 
 theta =  0.7853981633974483 
 node_a =  <class 'node.node'> 
 node_b =  <class 'node.node'>


0

In [7]:
#generating stiffness matrix for all elements

print(ele1.stiff())
print(ele2.stiff())
print(ele3.stiff())


[[ 1.  0. -1. -0.]
 [ 0.  0. -0. -0.]
 [-1. -0.  1.  0.]
 [-0. -0.  0.  0.]]
[[ 3.74939946e-33  6.12323400e-17 -3.74939946e-33 -6.12323400e-17]
 [ 6.12323400e-17  1.00000000e+00 -6.12323400e-17 -1.00000000e+00]
 [-3.74939946e-33 -6.12323400e-17  3.74939946e-33  6.12323400e-17]
 [-6.12323400e-17 -1.00000000e+00  6.12323400e-17  1.00000000e+00]]
[[ 0.5  0.5 -0.5 -0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [-0.5 -0.5  0.5  0.5]
 [-0.5 -0.5  0.5  0.5]]


In [8]:
#list of nodes and elements
nodes = [node1 , node2 , node3]
eles = [ele1 , ele2 ,ele3]


In [9]:
#generate global stiffness matrix 

def globalstiff(eles , num_nodes):
    #dimension of global matrix
    dim = 2 * num_nodes
    #generate and store the element-wise stiffness matrices
    SMs = [e.stiff() for e in eles]
    
    GK = np.zeros((dim,dim))
    for i in range(1,dim+1):
        for j in range(1,dim + 1):
            for k in range(0 , num_nodes):
                e = eles[k]
                if e.node_a == math.ceil(i/2):
                    if e.node_a == math.ceil(j/2):
                        GK[i-1][j-1] += SMs[k][(i+1)%2][(j+1)%2]
                    if e.node_b == math.ceil(j/2):
                        GK[i-1][j-1] += SMs[k][i%2][2+(j+1)%2]
                if e.node_b == math.ceil(i/2):
                    if e.node_a == math.ceil(j/2):
                        GK[i-1][j-1] += SMs[k][2+(i+1)%2][j%2]
                    if e.node_b == math.ceil(j/2):
                        GK[i-1][j-1] += SMs[k][2+(i+1)%2][2+(j+1)%2]
    return GK
            
    

In [10]:
GK = globalstiff(eles , 3)
GK

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [11]:
#create vector for node_dis and external_forces 

dis_list = []
for n in nodes:
    dis_list.append(n.dis_x)
    dis_list.append(n.dis_y)
    
dis = np.array(dis_list)
print(dis)

f_list = []
for n in nodes:
    f_list.append(n.f_x)
    f_list.append(n.f_y)
    
f = np.array(f_list)
print(f)


[ 0.  0.  0.  0. nan nan]
[     nan      nan      nan      nan  100000. -200000.]


In [12]:
#check for undetermined values in dis and create linear eqns

GK_dis =  copy.deepcopy(GK)
f_dis =  copy.deepcopy(f)

del_row = []
index_list = list(range(0,2*len(nodes)))
for i in range(0,2*len(nodes)):
    if dis_list[i] == 0:
        del_row.append(i)
        index_list.remove(i)
GK_dis = np.delete(GK_dis,del_row,0)
GK_dis = np.delete(GK_dis,del_row,1)
f_dis = np.delete(f_dis,del_row,0)
    
print(GK_dis)
print(f_dis)
print(index_list)
        
    
        

[[0. 0.]
 [0. 0.]]
[ 100000. -200000.]
[4, 5]


In [13]:
#check for undetermined values in force and create linear eqns

GK_dis =  copy.deepcopy(GK)
f_dis =  copy.deepcopy(f)

del_row = []
index_list = list(range(0,2*len(nodes)))
for i in range(0,2*len(nodes)):
    if dis_list[i] == 0:
        del_row.append(i)
        index_list.remove(i)
GK_dis = np.delete(GK_dis,del_row,0)
GK_dis = np.delete(GK_dis,del_row,1)
f_dis = np.delete(f_dis,del_row,0)
    
print(GK_dis)
print(f_dis)

[[0. 0.]
 [0. 0.]]
[ 100000. -200000.]


In [14]:
ans_dis = np.linalg.solve(GK_dis , f_dis)
ans_dis

LinAlgError: Singular matrix

In [ ]:
for i in range(0 , len(ans_dis)):
    dis[index_list[i]] = ans_dis[i]
dis

In [ ]:
ans_force = np.dot(GK,dis)
ans_force